# 05tools_4: Experiment Comparisons

Use the Python client to retrieve, review, and compare experiments from this series

### Prerequisites:
-  At least 1 of the notebooks in this series [05, 05a-05i]
    - Each of these notebooks creates an experiment.  Rerunning the notebooks creates additonal runs of the experiment.  The models created by each run are uploaded to the Vertex AI Model Registry with a version alias for the experiment run name and has labels for the experiment_name and run_name.

---
## Setup

inputs:

In [236]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [237]:
REGION = 'us-central1'
SERIES = '05'

packages:

In [238]:
from google.cloud import aiplatform
import pandas as pd
from IPython.display import HTML

clients:

In [239]:
aiplatform.init(project = PROJECT_ID, location = REGION)

---
## Experiments

Get a list of all experiments in this project:

In [240]:
experiments = aiplatform.Experiment.list()

Remove experiments not in the SERIES:

In [241]:
experiments = [e for e in experiments if e.name.split('-')[0:2] == ['experiment', SERIES]]

### Combine Runs from All Experiments in SERIES

Combine the runs from all experiments in SERIES into a single dataframe:

In [242]:
results = []
for experiment in experiments:
        results.append(experiment.get_data_frame())
        print(experiment.name)
results = pd.concat(results)

experiment-05-05i-classification-dnn
experiment-05-05h-classification-dnn
experiment-05-05g-classification-dnn
experiment-05-05f-classification-dnn
experiment-05-05e-classification-dnn
experiment-05-05c-classification-dnn
experiment-05-05b-classification-dnn
experiment-05-05d-classification-dnn
experiment-05-05a-classification-dnn
experiment-05-05-classification-dnn


In [243]:
results

,experiment_name,run_name,run_type,state,param.VAR_TARGET,param.var_split,param.hyperparameterTuning.link,param.hyperparameter.momentum,param.data_source,param.DATANAME,...,time_series_metric.val_accuracy,time_series_metric.train_auprc,param.customJobs.resource_name,param.customJobs.link,param.trainingPipelines.resource_name,param.customJobs.display_name,param.customJobs.tensorboard,param.trainingPipelines.display_name,param.MODEL_VERSIONED_RESOURCE_NAME,param.MODEL_DISPLAY_NAME
0,experiment-05-05i-classification-dnn,run-20220824133413-10,system.ExperimentRun,COMPLETE,Class,splits,https://console.cloud.google.com/ai/platform/l...,0.330811,bq://fraud.fraud_prepped,fraud,...,0.999221,0.999744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,experiment-05-05i-classification-dnn,run-20220824133413-8,system.ExperimentRun,COMPLETE,Class,splits,https://console.cloud.google.com/ai/platform/l...,0.157553,bq://fraud.fraud_prepped,fraud,...,0.999327,0.999750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,experiment-05-05i-classification-dnn,run-20220824133413-7,system.ExperimentRun,COMPLETE,Class,splits,https://console.cloud.google.com/ai/platform/l...,0.235337,bq://fraud.fraud_prepped,fraud,...,0.999256,0.999756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,experiment-05-05i-classification-dnn,run-20220824133413-6,system.ExperimentRun,COMPLETE,Class,splits,https://console.cloud.google.com/ai/platform/l...,0.060429,bq://fraud.fraud_prepped,fraud,...,0.999256,0.999773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,experiment-05-05i-classification-dnn,run-20220824133413-5,system.ExperimentRun,COMPLETE,Class,splits,https://console.cloud.google.com/ai/platform/l...,0.198008,bq://fraud.fraud_prepped,fraud,...,0.999292,0.999732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,experiment-05-05d-classification-dnn,run-20220821210919,system.ExperimentRun,COMPLETE,Class,splits,NaN,NaN,bq://fraud.fraud_prepped,fraud,...,0.999221,0.999692,projects/1026793852137/locations/us-central1/c...,https://console.cloud.google.com/vertex-ai/loc...,projects/1026793852137/locations/us-central1/t...,05d_fraud_20220821210919-custom-job,https://us-central1.tensorboard.googleusercont...,05d_fraud_20220821210919,NaN,NaN
0,experiment-05-05a-classification-dnn,run-20220823142225,system.ExperimentRun,COMPLETE,Class,splits,NaN,NaN,bq://fraud.fraud_prepped,fraud,...,0.999327,0.999721,projects/1026793852137/locations/us-central1/c...,https://console.cloud.google.com/vertex-ai/loc...,NaN,05a_fraud_20220823142225,https://us-central1.tensorboard.googleusercont...,NaN,NaN,NaN
1,experiment-05-05a-classification-dnn,run-20220821210837,system.ExperimentRun,COMPLETE,Class,splits,NaN,NaN,bq://fraud.fraud_prepped,fraud,...,0.999256,0.999692,projects/1026793852137/locations/us-central1/c...,https://console.cloud.google.com/vertex-ai/loc...,NaN,05a_fraud_20220821210837,https://us-central1.tensorboard.googleusercont...,NaN,NaN,NaN
0,experiment-05-05-classification-dnn,run-20220822002848,system.ExperimentRun,COMPLETE,Class,splits,NaN,NaN,bq://fraud.fraud_prepped,fraud,...,0.999044,0.999572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Get Links to all Experiment Runs with Vertex AI Training Jobs:
- Link to Job
- Link to TensorBoard for Job

In [244]:
# just names and job* columns
mask = results.columns.str.startswith(('param.trainingPipeline','param.customJobs','param.hyperparameterTuningJobs'))
mask[0:2] = True
HTML(results.loc[:,mask].to_html(render_links=True, escape=False))

,experiment_name,run_name,param.hyperparameterTuningJobs.display_name,param.customJobs.resource_name,param.customJobs.link,param.trainingPipelines.resource_name,param.customJobs.display_name,param.customJobs.tensorboard,param.trainingPipelines.display_name
0,experiment-05-05i-classification-dnn,run-20220824133413-10,05i_fraud_20220824133413,NaN,NaN,NaN,NaN,NaN,NaN
1,experiment-05-05i-classification-dnn,run-20220824133413-8,05i_fraud_20220824133413,NaN,NaN,NaN,NaN,NaN,NaN
2,experiment-05-05i-classification-dnn,run-20220824133413-7,05i_fraud_20220824133413,NaN,NaN,NaN,NaN,NaN,NaN
3,experiment-05-05i-classification-dnn,run-20220824133413-6,05i_fraud_20220824133413,NaN,NaN,NaN,NaN,NaN,NaN
4,experiment-05-05i-classification-dnn,run-20220824133413-5,05i_fraud_20220824133413,NaN,NaN,NaN,NaN,NaN,NaN
5,experiment-05-05i-classification-dnn,run-20220824133413-4,05i_fraud_20220824133413,NaN,NaN,NaN,NaN,NaN,NaN
6,experiment-05-05i-classification-dnn,run-20220824133413-3,05i_fraud_20220824133413,NaN,NaN,NaN,NaN,NaN,NaN
7,experiment-05-05i-classification-dnn,run-20220824133413-2,05i_fraud_20220824133413,NaN,NaN,NaN,NaN,NaN,NaN
8,experiment-05-05i-classification-dnn,run-20220824133413-1,05i_fraud_20220824133413,NaN,NaN,NaN,NaN,NaN,NaN
9,experiment-05-05i-classification-dnn,run-20220824133413-9,05i_fraud_20220824133413,NaN,NaN,NaN,NaN,NaN,NaN


### Hyperlinks To TensorBoard for Experiments and Comparison Of All Experiments

Get a hyperlink to the TensorBoard for each experiment:

In [245]:
tensorboards = []
for experiment in experiments:
    tboard = experiment.get_backing_tensorboard_resource().resource_name.replace('/', '+')
    tensorboards.append([experiment.name, f"https://{REGION}.tensorboard.googleusercontent.com/experiment/{tboard}+experiments+{experiment.name}"])
tensorboards = pd.DataFrame(tensorboards, columns = ['Experiment', 'TensorBoard Link'])    

In [246]:
HTML(tensorboards.to_html(render_links=True, escape=False))

,Experiment,TensorBoard Link
0,experiment-05-05i-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+experiment-05-05i-classification-dnn
1,experiment-05-05h-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+experiment-05-05h-classification-dnn
2,experiment-05-05g-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+experiment-05-05g-classification-dnn
3,experiment-05-05f-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+experiment-05-05f-classification-dnn
4,experiment-05-05e-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+experiment-05-05e-classification-dnn
5,experiment-05-05c-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+experiment-05-05c-classification-dnn
6,experiment-05-05b-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+experiment-05-05b-classification-dnn
7,experiment-05-05d-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+experiment-05-05d-classification-dnn
8,experiment-05-05a-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+experiment-05-05a-classification-dnn
9,experiment-05-05-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+experiment-05-05-classification-dnn


Compare all experiments in SERIES with TensorBoard:

In [247]:
compare_link = f"https://{REGION}.tensorboard.googleusercontent.com/compare/"
for e, experiment in enumerate(experiments):
    if e>0: compare_link += ','
    tboard = experiment.get_backing_tensorboard_resource().resource_name.split('/')
    compare_link += f"{e+1}-{experiment.name}:{'+'.join(tboard[1::2])}+{experiment.name}"
print(compare_link)

https://us-central1.tensorboard.googleusercontent.com/compare/1-experiment-05-05i-classification-dnn:1026793852137+us-central1+3514619704511561728+experiment-05-05i-classification-dnn,2-experiment-05-05h-classification-dnn:1026793852137+us-central1+3514619704511561728+experiment-05-05h-classification-dnn,3-experiment-05-05g-classification-dnn:1026793852137+us-central1+3514619704511561728+experiment-05-05g-classification-dnn,4-experiment-05-05f-classification-dnn:1026793852137+us-central1+3514619704511561728+experiment-05-05f-classification-dnn,5-experiment-05-05e-classification-dnn:1026793852137+us-central1+3514619704511561728+experiment-05-05e-classification-dnn,6-experiment-05-05c-classification-dnn:1026793852137+us-central1+3514619704511561728+experiment-05-05c-classification-dnn,7-experiment-05-05b-classification-dnn:1026793852137+us-central1+3514619704511561728+experiment-05-05b-classification-dnn,8-experiment-05-05d-classification-dnn:1026793852137+us-central1+3514619704511561728

## Rank Experiment Runs on a Metric

In [248]:
def ranker(metric = 'metric.test_auprc'):
    ranks = results[['experiment_name', 'run_name', metric]].copy().reset_index(drop = True)
    ranks['rank'] = ranks[metric].rank(method='dense', ascending = False)
    return ranks.sort_values(by = ['rank'])
    
ranks = ranker()

In [249]:
ranks

,experiment_name,run_name,metric.test_auprc,rank
39,experiment-05-05g-classification-dnn,run-20220823193746-6,0.999812,1.0
55,experiment-05-05c-classification-dnn,run-20220821211710,0.999811,2.0
21,experiment-05-05h-classification-dnn,run-20220823232202-8,0.999766,3.0
29,experiment-05-05h-classification-dnn,run-20220823232202-14,0.999765,4.0
44,experiment-05-05g-classification-dnn,run-20220822211125-5,0.999765,5.0
...,...,...,...,...
16,experiment-05-05h-classification-dnn,run-20220823232202-13,0.999368,56.0
10,experiment-05-05h-classification-dnn,run-20220823232202-20,0.999336,57.0
17,experiment-05-05h-classification-dnn,run-20220823232202-12,0.999332,58.0
20,experiment-05-05h-classification-dnn,run-20220823232202-9,0.999272,59.0
